# Project 2：NASA Data Acquisation, Visualization, and Analysis

In [ ]:
# The code makes sure that once there is change in the 'src/' folder, the 
# change will be automatically reloaded in the notebook.
%reload_ext autoreload
%autoreload 2
%aimport src

### Task 1: Understanding the NASA API and Data Collection

- Register for a NASA API key and understand the different types of data that the API provides.
- Run the Python script below to fetch data about **Near Earth Objects (NEOs)** from the NASA API for a years data.
- Extract and understand the different pieces of data provided for each NEO.

In [ ]:
import requests
import time
from datetime import datetime, timedelta
from getpass import getpass

# Set your NASA API KEY, this step asks you to enter your API KEY.
# (The input box may be float in the top on your editor.)
api_key = getpass()

In [ ]:
# Set the start and end dates for the data you want to fetch
start_date = datetime.strptime('2022-01-01', '%Y-%m-%d')
end_date = start_date + timedelta(days=365)  # 1 year later

# Initialize a list to store the data
data = []

# Fetch data from the NASA API 7 days at a time
# The introduction of the API is on https://api.nasa.gov, under "Browse APIs" -> "Asteroids NeoWs"
# You can look into the example query in the link below to see what the data look like:
# https://api.nasa.gov/neo/rest/v1/feed?start_date=2015-09-07&end_date=2015-09-08&api_key=DEMO_KEY
current_date = start_date
while current_date < end_date:
    next_date = min(current_date + timedelta(days=7), end_date)
    response = requests.get(f'https://api.nasa.gov/neo/rest/v1/feed?start_date={current_date.strftime("%Y-%m-%d")}&end_date={next_date.strftime("%Y-%m-%d")}&api_key={api_key}')
    data.append(response.json())
    current_date = next_date
    time.sleep(1)  # To avoid hitting the rate limit

# Now 'data' contains the NEO data for the 1-year period


In [ ]:
# Check the date coverage of your data.
dates_contained_in_data = []
for d in data:
    dates_contained_in_data += list(d['near_earth_objects'].keys())

print(sorted(dates_contained_in_data))

In [ ]:
from src.utils import get_a_random_chunk_property

In [ ]:
get_a_random_chunk_property(data)

For the remaining tasks, you have to organize the data as pd.DataFrame so as to suit the specific need in each task. This part may require a considerably amount of efforts, which is normal in data science and analytics works.

### Task 2: Data Analysis

- Calculate the average size of the NEOs for each day.
- Determine the proportion of NEOs that are potentially hazardous.
- Find the NEO with the closest approach distance for each day.
- Use statistical methods to analyze the data. For example, calculate the mean, median, mode, and standard deviation of the NEO sizes. Determine if the size of a NEO is correlated with whether it is potentially hazardous.

In [29]:
import json
import pandas as pd
import numpy as np
from scipy.stats import mode
with open('data.json') as file:
    json_data = json.load(file)

flattened_data = []
for entry in json_data:
    near_earth_objects = entry["near_earth_objects"]

    for date, objects in near_earth_objects.items():
        for obj in objects:
            obj["close_approach_date"] = date
            flattened_data.append(obj)

df = pd.json_normalize(flattened_data)

# Convert the date column to datetime type
df['close_approach_date'] = pd.to_datetime(df['close_approach_date'])

# Define the list of keys for diameter columns
diameter_keys = ['estimated_diameter.kilometers.estimated_diameter_min', 'estimated_diameter.kilometers.estimated_diameter_max']

# Extract the required columns and convert them to numeric type
df[diameter_keys] = df[diameter_keys].apply(pd.to_numeric)

# Group by date and calculate the average diameter for each day
average_diameter = df.groupby('close_approach_date')[diameter_keys].mean()

print(average_diameter)


# Calculate the proportions of potentially hazardous near_earth_objects
proportions = df['is_potentially_hazardous_asteroid'].value_counts(normalize=True)

print(proportions)

# Calculate the correlation between NEO sizes and their potentially hazardous nature
correlation = df[diameter_keys].corrwith(df['is_potentially_hazardous_asteroid'].astype(int))

print(correlation)

# Calculate the mean, median, mode, and standard deviation of the NEO sizes
mean_diameter = np.mean(df[diameter_keys].values)
median_diameter = np.median(df[diameter_keys].values)
mode_diameter = mode(df[diameter_keys].values, axis=None, keepdims=True)[0][0]
std_diameter = np.std(df[diameter_keys].values)

print("Mean diameter:", mean_diameter)
print("Median diameter:", median_diameter)
print("Mode diameter:", mode_diameter)
print("Standard deviation of diameter:", std_diameter)


                     estimated_diameter.kilometers.estimated_diameter_min  \
close_approach_date                                                         
2022-01-01                                                    0.101401      
2022-01-02                                                    0.070013      
2022-01-03                                                    0.017416      
2022-01-04                                                    0.049554      
2022-01-05                                                    0.109484      
...                                                                ...      
2022-12-28                                                    0.077546      
2022-12-29                                                    0.116757      
2022-12-30                                                    0.084969      
2022-12-31                                                    0.032318      
2023-01-01                                                    0.188559      

### Task 3: Data Visualization Part A

- Create a line plot of the number of NEOs per week.
- Create a histogram of the distribution of NEO sizes.
- Create a bar plot of the average NEO size per week.
- Use a library like Seaborn to create more complex visualizations, such as a box plot of the NEO sizes or a heat map of the number of NEOs per week. **Be creative**!

In [ ]:
# Write your code

### Task 4: Data Visualization Part B

- Create a pie chart of the proportion of hazardous vs non-hazardous NEOs.
- Create a scatter plot of the correlation between NEO size and close approach distance.
- Customize the appearance of your plots (e.g., colors, labels, titles).
- Create interactive visualizations using a library like Plotly. For example, create an interactive scatter plot where you can hover over each point to see more information about the NEO. **Be creative!**

In [ ]:
# Write your code

### Task 5: Interpretation of Results

- Interpret the results of your data visualization in part A and B. 
- What insights can you gain about NEOs from your results? Summarizing your findings.
- Use your findings to make predictions or recommendations. For example, if you found that larger NEOs are more likely to be potentially hazardous, you could recommend that more resources be allocated to tracking large NEOs. **Be creative!**
- Identify, understand, and explain one scientific paper, on a clustering or classification method of relevance that could help Task 5. You don't have to implement it, you just need to justify in this notebook why the method in the scientific paper could contribute in analysis or interpretation of the results.

In [ ]:
# Write your code

### Task 6: Presentation and Documentation

- Make this project as part of your presentation, **using beamer in LaTeX**. 
- This should include an overview of your work, the results of your data analysis, and the insights you gained from your results.